下载对应版本的transformers

In [ ]:
!pip install -q transformers==4.41.2
!pip install -q accelerate==0.30.1
!pip install -q peft==0.11.1
!pip install -q trl==0.8.6
!pip install -q datasets==2.19.0
!pip install -q bitsandbytes
!pip install -q fsspec==2025.3.0
!pip install -q gcsfs==2025.3.0
!pip install openai -q

由于要读取json文件的内容，因此需要在colab上将drive挂载到该python notebook中

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

查看一下GPU信息

In [ ]:
!nvidia-smi

导入需要的库

In [ ]:
from datasets import load_dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, TrainingArguments
from peft import LoraConfig, prepare_model_for_kbit_training, AutoPeftModelForCausalLM
from trl import SFTTrainer
from openai import OpenAI
# import csv
import json

加载数据

In [ ]:
dataset_path = "/content/drive/MyDrive/java_sft/data/java_interview.jsonl"
raw_dataset = load_dataset("json", data_files=dataset_path)

# 默认会有一个 "train" split，这里再划分出验证集
dataset = raw_dataset["train"].train_test_split(test_size=0.1, seed=42)
dataset


加载模型（Qwen/Qwen2.5-3B-Instruct）

In [ ]:
model_name = "Qwen/Qwen2.5-3B-Instruct"  # 比原来的 Base 版更适合做 Chat 微调

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)

# 有些 Qwen 没显式 pad_token，这里兜底一下
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    load_in_4bit=True,     # 4bit 量化
    device_map="auto",
    trust_remote_code=True
)
model = prepare_model_for_kbit_training(model)


构建 LoRA 微调配置

In [ ]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    # target_modules=["q_proj", "v_proj"],
    target_modules=["qkv_proj", "o_proj"],
    bias="none",
    task_type="CAUSAL_LM"
)

构造 SFTTrainer（核心训练）

In [ ]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/java_sft/qwen-java-sft",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    logging_steps=20,
    num_train_epochs=2,
    save_steps=200,
    learning_rate=2e-4,
    bf16=True,
)

# def format_example(example):
#     prompt = (
#         f"<|im_start|>system\n{example['system']}<|im_end|>\n"
#         f"<|im_start|>user\n{example['input']}<|im_end|>\n"
#         f"<|im_start|>assistant\n{example['output']}<|im_end|>"
#     )
#     return [prompt]

def format_example(batch):
    results = []

    # batch["system"] 是一个 list，例如 ["你是面试官", "你是面试官", ...]
    for sys_msg, usr_msg, asst_msg in zip(batch["system"], batch["input"], batch["output"]):

        messages = [
            {"role": "system", "content": sys_msg},
            {"role": "user", "content": usr_msg},
            {"role": "assistant", "content": asst_msg},
        ]

        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False
        )

        results.append(text)

    return results    # <-- 必须是 list[str]


trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    dataset_text_field=None,
    max_seq_length=1024,
    formatting_func=format_example,
    args=training_args,
    peft_config=lora_config
)


开始训练

In [ ]:
trainer.train()

保存 LoRA 模型

In [ ]:
# 这里保存的是 LoRA + 基座的 PeftModel
trainer.model.save_pretrained("/content/drive/MyDrive/java_sft/qwen-java-sft")
tokenizer.save_pretrained("/content/drive/MyDrive/java_sft/qwen-java-sft")

推理测试（加载 LoRA 模型）

In [ ]:
# ---------------------------
# 1. 加载微调前模型（Base Model）
# ---------------------------
base_model_name = model_name  # 和上面保持一致

base_tokenizer = AutoTokenizer.from_pretrained(
    base_model_name,
    trust_remote_code=True
)
if base_tokenizer.pad_token is None:
    base_tokenizer.pad_token = base_tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype="auto",
    device_map="auto",
    trust_remote_code=True
)

# ---------------------------
# 2. 加载微调后模型（LoRA + SFT）
# ---------------------------
ft_model_path = "/content/drive/MyDrive/java_sft/qwen-java-sft"

ft_tokenizer = AutoTokenizer.from_pretrained(
    ft_model_path,
    trust_remote_code=True
)
if ft_tokenizer.pad_token is None:
    ft_tokenizer.pad_token = ft_tokenizer.eos_token

# 关键：使用 AutoPeftModelForCausalLM 加载带 LoRA 的模型
ft_model = AutoPeftModelForCausalLM.from_pretrained(
    ft_model_path,
    torch_dtype="auto",
    device_map="auto"
)

# base_pipe = pipeline("text-generation", model=base_model, tokenizer=base_tokenizer)
# ft_pipe = pipeline("text-generation", model=ft_model, tokenizer=ft_tokenizer)
ft_model = ft_model.merge_and_unload()

定义答案生成与答案对比函数

In [ ]:
def generate_answer(pipe, tokenizer, question, max_new_tokens=256):
    messages = [
        {"role": "system", "content": "你是一名 Java 面试官。"},
        {"role": "user", "content": question}
    ]

    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # 先拿到 prompt 的长度
    # inputs = tokenizer(prompt, return_tensors="pt").to(pipe.model.device)
    # prompt_len = inputs["input_ids"].shape[1]
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    prompt_len = inputs["input_ids"].shape[1]

    # outputs = pipe(
    #     prompt,
    #     max_new_tokens=max_new_tokens,
    #     do_sample=False,   # 面试场景更追求稳定，可以先关掉 sampling
    #     pad_token_id=tokenizer.eos_token_id
    # )[0]["generated_text"]

    # # 重新用 tokenizer 编码解码，截取「新生成的」部分
    # output_ids = tokenizer(outputs, return_tensors="pt")["input_ids"][0]
    # new_tokens = output_ids[prompt_len:]  # 只取新生成 token

    # answer = tokenizer.decode(new_tokens, skip_special_tokens=True).strip()
    # return answer
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,    # 不采样 → 稳定可复现
        pad_token_id=tokenizer.eos_token_id
    )

    new_tokens = outputs[0][prompt_len:]
    answer = tokenizer.decode(new_tokens, skip_special_tokens=True).strip()

    return answer

# def compare_answer(question, max_new_tokens=256):
#     base_answer = generate_answer(base_pipe, base_tokenizer, question, max_new_tokens)
#     ft_answer = generate_answer(ft_pipe, ft_tokenizer, question, max_new_tokens)

#     # print("========【微调前】========")
#     # print(base_answer)
#     # print("\n========【微调后】========")
#     # print(ft_answer)

#     return base_answer, ft_answer
def compare_answer(question, max_new_tokens=256):
    base_answer = generate_answer(base_model, base_tokenizer, question, max_new_tokens)
    ft_answer = generate_answer(ft_model, ft_tokenizer, question, max_new_tokens)

    return base_answer, ft_answer


验证及对比结果

In [ ]:
question = "Redis的持久化策略有两种，分别是什么？以及两者的区别是什么？"
res = compare_answer(question, 1024)
print("========【微调前】========")
print(res[0])
print("\n========【微调后】========")
print(res[1])

批量验证结果并输出为JSON

In [ ]:
# ======================
# 设置 DeepSeek API KEY
# ======================
client = OpenAI(
    api_key="sk-739e4ade526e4316afe7789f040e93bf",
    base_url="https://api.deepseek.com"
)

# ======================
# 评分 Rubric
# ======================
RUBRIC = """
你是一名专业的 Java 技术面试官与评分裁判。
请严格依据以下评分标准对“模型回答”进行客观、公正的评分，总分 100 分。

=====================
【评分维度与细则（详细说明）】
=====================

1. 技术正确性（0–40 分）
- 回答是否准确描述了概念、原理、机制或流程？
- 是否存在技术性错误、误解或不严谨之处？
- 是否能覆盖问题核心点？
- 若存在关键性错误，分数大幅降低。

评分区间示例：
- 35–40：完全正确、内容严谨、无错误，技术细节准确。
- 25–34：整体正确，但存在轻微遗漏或浅层不严谨。
- 10–24：部分正确，但有明显理解偏差。
- 0–9：技术上错误严重、明显误解问题。

------------------------------------------

2. 简洁性（0–20 分）
- 是否避免冗余或长篇大论？
- 是否以面试官喜欢的方式快速给出重点？

评分区间示例：
- 18–20：非常简洁，关键点一句话即可理解。
- 13–17：基本简洁，但稍有赘述。
- 6–12：存在较多废话或不必要的解释。
- 0–5：内容冗长、偏科普文章风格。

------------------------------------------

3. 结构化程度（0–15 分）
- 回答是否具有清晰结构（如“概念 → 原理 → 优缺点 → 场景”）？
- 是否使用了条列式、分点式、层次结构？

评分区间示例：
- 13–15：结构优秀，逻辑强，可直接作为面试标准答案。
- 9–12：有一定结构，但不够突出。
- 4–8：结构松散但可理解。
- 0–3：缺乏结构，思路混乱。

------------------------------------------

4. 面试场景适配度（0–15 分）
- 回答是否像面试者应答？而不是像博客、百科？
- 是否避免过度展开？
- 是否符合真实 Java 后端面试环境？

评分区间示例：
- 13–15：完全符合面试风格，重点明确、直击痛点。
- 9–12：大体符合，但稍微啰嗦或偏文档风格。
- 4–8：偏科普写法，不像面试交流。
- 0–3：完全不符合面试语境。

------------------------------------------

5. 专业深度（0–10 分）
- 是否体现出对技术的理解深度？
- 是否涉及底层细节、性能影响、设计动机、使用场景？

评分区间示例：
- 9–10：展现深入洞察，体现资深工程师水平。
- 6–8：中等深度，覆盖表层 + 中层内容。
- 3–5：仅停留在表层定义。
- 0–2：内容非常浅显，没有专业性。

=====================
【输出要求】
请只输出 JSON，格式如下：

{
  "technical_correctness": x,
  "conciseness": x,
  "structure": x,
  "interview_appropriateness": x,
  "professional_depth": x,
  "overall": x
}

=====================
请严格遵守以上规则进行评分，不要加入额外解释。
"""

# ======================
# 评分函数
# ======================
def score_answer(question, answer):

    user_prompt = f"""
【问题】
{question}

【回答】
{answer}

请依据评分细则严格评分，并输出必须JSON。
"""

    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": RUBRIC},
            {"role": "user", "content": user_prompt}
        ]
    )

    content = response.choices[0].message.content

    try:
        return json.loads(content)
    except:
        print("JSON 解析失败：", content)
        return {}


# ======================
# 读取问题并评估
# ======================
input_file = "/content/drive/MyDrive/java_sft/data/test_questions_shuffled_50.txt"
output_jsonl = "/content/drive/MyDrive/java_sft/data/evaluation_results_deepseek.jsonl"
output_csv = "/content/drive/MyDrive/java_sft/data/evaluation_results_deepseek.csv"

results = []

with open(input_file, "r", encoding="utf-8") as f:
    questions = [q.strip() for q in f if q.strip()]

print(f"Loaded {len(questions)} questions.")
ft_count = 0

for idx, question in enumerate(questions, start=1):
    print(f"\n===== Processing Q{idx}: {question} =====")

    base_answer, ft_answer = compare_answer(question, max_new_tokens=512)

    base_score = score_answer(question, base_answer)
    ft_score = score_answer(question, ft_answer)

    winner = "ft" if ft_score.get("overall", 0) > base_score.get("overall", 0) else "base"

    if ft_score.get("overall", 0) > base_score.get("overall", 0):   ft_count += 1

    entry = {
        "question": question,
        "base_answer": base_answer,
        "ft_answer": ft_answer,
        "base_score": base_score,
        "ft_score": ft_score,
        "winner": winner
    }

    results.append(entry)

    with open(output_jsonl, "a", encoding="utf-8") as fjson:
        fjson.write(json.dumps(entry, ensure_ascii=False) + "\n")

# 保存 CSV
# with open(output_csv, "w", newline="", encoding="utf-8") as fcsv:
#     writer = csv.writer(fcsv)
#     writer.writerow(["Question", "Base Overall", "FT Overall", "Winner"])

#     for r in results:
#         writer.writerow([
#             r["question"],
#             r["base_score"].get("overall", 0),
#             r["ft_score"].get("overall", 0),
#             r["winner"]
#         ])

print("\n评估完成！")
print("JSONL:", output_jsonl)
print(f"ft wins: {ft_count} times")
# print("CSV:", output_csv)
